In [3]:
# Import dependencies
# pip install required only first time when this file is run on a system
# !pip install wrds 
import wrds
import pandas as pd
import sqlalchemy 
import psycopg2 


## Define and setup starting database 

In [4]:
conn = psycopg2.connect(dbname='wrds', 
                        user='', 
                        host='wrds-pgdata.wharton.upenn.edu', 
                        port=9737,
                        password='')

In [5]:
# Upload the local files to pandas dataframe
inputfile = pd.read_csv("tickers.csv")
mappingtable = pd.read_csv("tickermapping.csv")

In [6]:
# Yet to check whether the same ticker / fund no is used in the history which might creates errors
# For now, Keeping only the desired columns and Removing multiples to create unique matches 
mappingtable = mappingtable.drop(['summary_period2', 'caldt', 'fund_name'], axis=1)
mappingtable = mappingtable.dropna() #there were NaN values in ticker for many of them
mappingtable = mappingtable.drop_duplicates(subset=['ticker'],keep = 'last')
mappingtable = mappingtable.reset_index(drop= True)
mappingtable.sort_values(by='ticker',inplace=True)

In [7]:
# Merging based on the tickers- one to one didnt happen until I dropped all duplicates of ticker 
## -- needs to checked-- currently it keeps the last observation sorted on date
required_mfs = pd.merge(left = inputfile, right = mappingtable, how = 'left',on = 'ticker', validate = "one_to_one")

In [8]:
# Check if all tickers were found
print(inputfile.shape)
print(required_mfs.shape)
# this matches the input file shape so all the tickers were found.

(91, 1)
(91, 2)


In [10]:
#Create a list of crsp_fundno to be used
fund_number=list(required_mfs['crsp_fundno'].values)
len(fund_number)

91

In [33]:
# # Checking connection and statement with single query, keeping this if required to run
# mfdaily = pd.read_sql_query("""SELECT * FROM crspq.daily_nav_ret where crsp_fundno =3173;""", conn)

# # mfdaily = db.raw_sql("SELECT * FROM crspq.daily_nav_ret where crsp_fundno =3173")
# mfdaily.head()

In [12]:
#Create an empty dataframe to add all values 
dailyreturns = pd.DataFrame()

for fundno in fund_number:
    query = f'SELECT * FROM crspq.daily_nav_ret WHERE crsp_fundno = {fundno};'
    mfdaily =  pd.read_sql_query(query,conn)
    dailyreturns = dailyreturns.append(mfdaily)
    print(mfdaily.shape) #just to check if it's returning something
    

dailyreturns.shape

(3817, 4)
(3165, 4)
(5240, 4)
(5240, 4)
(3186, 4)
(4383, 4)
(5240, 4)
(5161, 4)
(5240, 4)
(5240, 4)
(5240, 4)
(4778, 4)
(4108, 4)
(5240, 4)
(5240, 4)
(5240, 4)
(5240, 4)
(3678, 4)
(4193, 4)
(4599, 4)
(4745, 4)
(5240, 4)
(5240, 4)
(5240, 4)
(5240, 4)
(5240, 4)
(5240, 4)
(5240, 4)
(5240, 4)
(4520, 4)
(5240, 4)
(4028, 4)
(5240, 4)
(5031, 4)
(5240, 4)
(5240, 4)
(4007, 4)
(5240, 4)
(5240, 4)
(5017, 4)
(5240, 4)
(3755, 4)
(5240, 4)
(4035, 4)
(5240, 4)
(3185, 4)
(5240, 4)
(5240, 4)
(5240, 4)
(5240, 4)
(5240, 4)
(5240, 4)
(5240, 4)
(3021, 4)
(2958, 4)
(2934, 4)
(2764, 4)
(2764, 4)
(2888, 4)
(2645, 4)
(2406, 4)
(2272, 4)
(2243, 4)
(2142, 4)
(2132, 4)
(2093, 4)
(2090, 4)
(2095, 4)
(1987, 4)
(1943, 4)
(1840, 4)
(1789, 4)
(1781, 4)
(1692, 4)
(1650, 4)
(1635, 4)
(1575, 4)
(1423, 4)
(1388, 4)
(1378, 4)
(1283, 4)
(1240, 4)
(1044, 4)
(796, 4)
(754, 4)
(426, 4)
(329, 4)
(310, 4)
(251, 4)
(251, 4)
(136, 4)


(321899, 4)

In [13]:
# Save files to single file
# !pip install xlsxwriter #required to run only first time
import xlsxwriter

writer = pd.ExcelWriter('mfreturns.xlsx', engine='xlsxwriter')

dailyreturns.to_excel(writer, sheet_name='returns')
required_mfs.to_excel(writer, sheet_name='fundno_ticker')


writer.save()

### Creating some visualizations

In [16]:
dailyreturns.head()

,crsp_fundno,caldt,dnav,dret
0,3173.0,2004-04-30,10.03,NaN
1,3173.0,2004-05-03,10.02,-0.000997
2,3173.0,2004-05-04,10.02,0.000000
3,3173.0,2004-05-05,10.01,-0.000998
4,3173.0,2004-05-06,10.01,0.000000


In [17]:
import matplotlib as plt